# Importar librerias necesarias

In [1]:
import pandas as pd
import os

Wojciech Hardy del Instituto de investigación estructural, en conjunto con la Facultad de Economía de la Universidad de Warsaw, desarrollaron una correlativa para el SOC-10 a 6 dígitos con el CIUO-08 a 4 dígitos (Clasificación Internacional Uniforme de Ocupaciones)

https://ibs.org.pl/en/resources/occupation-classifications-crosswalks-from-onet-soc-to-isco/

Este primer insumo es el punto de partida para la correlativa de SOC a CIUO-08.

In [2]:
reader=pd.read_stata("soc10_isco08.dta",chunksize=100000)
df= pd.DataFrame()
for itm in reader:
    df=df.append(itm)


In [92]:
df.shape

(1131, 2)

la Organización Internacional del Trabajo3 desarrolló una correlativa del CIUO-08 y el CIUO-88, ambas a 4 dígitos. Basado en esta correspondencia, se cruzó ésta con la base del inciso número 1, arrojando una base insumo de la forma SOC-10 y CIUO-88.

https://www.ilo.org/public/spanish/bureau/stat/isco/isco08/index.htm


In [4]:
df1=pd.read_excel("corrtab08-88.xls")

In [93]:
df1.shape

(673, 6)

Luego de transcribir a una base de datos la correlativa realizada por DANE del código CIUO-88 de 4 dígitos a la CNO-70 a cuatro dígitos, la siguiente es la base resultante:


In [11]:
cno=pd.read_excel("CNO 70 GEIH.xlsx",5)

In [94]:
cno.shape

(345, 2)

Este primer cruce es la correlativa para pasar de SOC-10 a CIIUO-88:

In [6]:
cruce=pd.merge(df, df1[['Código CIUO-08', 'Código CIUO-88']], left_on='isco08', right_on='Código CIUO-08', how='inner').drop_duplicates().reset_index(drop=True)

In [7]:
cruce.shape

(1703, 4)

Una vez realizado el cruce, se debe quitan los duplicados sobre la correlativa deseada:

In [8]:
cruce1=cruce[['soc10','Código CIUO-88']].drop_duplicates().reset_index(drop=True)

Agrupar el código CNO-70 a dos dígitos y relacionar el código CIUO-88 a 4 dígitos correspondiente. En este punto, se realiza una limpieza de la agrupación en donde se eliminen duplicados y ocupaciones del CIUO-88 a cuatro dígitos que no corresponden a la subcategoría de CNO-70 a los dos dígitos que usa el DANE en la GEIH.

Cruzar la base cruce1 por la llave CIUO-88 con la base del inciso anterior para obtener una correlativa del código CNO-70 a dos dígitos con el SOC-10 a 6 dígitos.

In [13]:
cruce2=pd.merge(cno, cruce1, left_on='CIUO', right_on='Código CIUO-88', how='inner').drop_duplicates().reset_index(drop=True)

In [14]:
cruce2.shape

(1443, 4)

In [15]:
cruce3=cruce2[['DANE', 'soc10']].drop_duplicates().reset_index(drop=True)

Cruzar la base de probabilidades de riesgo de automatización del paper de Frey & Osborne (2013) con la base del inciso anterior, por la llave SOC-10 a 6 dígitos. Luego, agrupar a nivel de CNO-70 y dejar el promedio de probabilidades relacionadas a los códigos SOC-10 en la correlativa de la base cruce3.

In [18]:
frey=pd.read_excel("CNO 70 GEIH.xlsx",1)

In [19]:
frey['SOC code']=frey['SOC code'].map(lambda x:x.replace("-",""))
frey['SOC code']=frey['SOC code'].astype(int)

In [96]:
cruce4=pd.merge(cruce3, frey, left_on='soc10', right_on='SOC code', how='inner').drop_duplicates().reset_index(drop=True)

In [97]:
cruce4.shape

(1021, 7)

Se exporta la correlativa y se confirma que las ocupaciones no genere duplicados.

In [98]:
cruce4.to_excel("base_frey.xlsx", index=False)

Una vez realizado el cruce, se debe revisar las ocupaciones que a nivel SOC-6 se repite a nivel de la ocupacion DANE. Esto debido a que la ocupacion dane puede tener cierto conjunto de tareas que la ocupacion SOC-6 tambien comparte pero no en la mayoria. Por tanto, se dejaran solo las ocupaciones que comparten la mayor parte de sus tareas.

In [67]:
cruce5=pd.read_excel("base_prob_frey.xlsx")

In [80]:
cruce6=pd.merge(cruce4,cruce5[['DANE','soc10']], how='left', on=['soc10', 'DANE'], indicator=True)

In [87]:
cruce6=cruce6[cruce6['_merge']=='both']

In [89]:
cruce6.shape

(467, 8)

In [90]:
bla=cruce6[['DANE','Probability']].groupby(['DANE'])
counts = bla.size().to_frame(name='counts')
cosechas5=(counts
  .join(bla.agg({'Probability': 'mean'}))          
  .reset_index())

In [91]:
cosechas5

,DANE,counts,Probability
0,1,7,0.385000
1,2,19,0.072421
2,3,7,0.337286
3,4,14,0.207186
4,5,10,0.369900
...,...,...,...
74,95,1,0.940000
75,96,3,0.920000
76,97,1,0.420000
77,98,4,0.820000


In [61]:
cosechas5.to_excel("base_prob_frey1_.xlsx",  index=False)

# Revision de representatividad

In [99]:
import pandas as pd
df= pd.read_gbq("Dane.consolidado_geih_2019_2007_oficio", "fabled-opus-301614")

In [100]:
df.shape

(1223540, 10)

In [101]:
df[df['Probability'].isnull()][['OFICIO']].drop_duplicates()

,OFICIO
0,0
1136,52
3907,70
5406,49


In [102]:
import numpy as np
df['DESCRIPCION1']=np.where(df['Probability'].isnull(),"Vacio", df['DESCRIPCION'])

In [104]:
pd.crosstab(df['DESCRIPCION1'],df["ano"], aggfunc="sum", dropna=False, values=df['fex_12'], normalize='columns')

ano,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
DESCRIPCION1,,,,,,,,,,,,,
"Actividades inmobiliarias, empresariales y de alquiler",0.054531,0.063019,0.063540,0.062169,0.064227,0.066136,0.069871,0.071706,0.075414,0.078206,0.081010,0.077217,0.074235
Administración pública y defensa; seguridad social de afiliación obligatoria,0.028560,0.027942,0.028914,0.027825,0.026717,0.027035,0.027883,0.030837,0.029390,0.030180,0.029571,0.031242,0.030902
"Agricultura, ganadería, caza y silvicultura",0.177279,0.176342,0.179392,0.178836,0.174248,0.168024,0.162497,0.156882,0.155058,0.155698,0.160342,0.160867,0.153667
"Comercio al por mayor y al por menor; reparación de vehículos automotores, motocicletas, efectos personales y enseres domésticos",0.206383,0.210809,0.207876,0.208797,0.209200,0.207530,0.210101,0.206142,0.204002,0.204851,0.202431,0.200874,0.200894
Construcción,0.052529,0.050848,0.052430,0.054380,0.057080,0.059931,0.057844,0.060446,0.062347,0.063050,0.061129,0.062208,0.067976
Educación,0.047056,0.045674,0.041317,0.041416,0.040934,0.040498,0.042200,0.041048,0.040257,0.041212,0.041046,0.040684,0.041248
Explotación de minas y canteras,0.010430,0.011375,0.009861,0.010921,0.012100,0.011267,0.010594,0.009930,0.009210,0.008144,0.008298,0.009282,0.008899
"Hogares privados con servicio doméstico, actividades no diferenciadas de hogares privados como productores de bienes o servicio para uso propio",0.047730,0.036794,0.037376,0.036505,0.035073,0.035378,0.033772,0.033250,0.033338,0.030359,0.030280,0.029559,0.031004
Hoteles y restaurantes,0.044286,0.046304,0.050676,0.053093,0.054619,0.059905,0.064171,0.066167,0.070254,0.073800,0.069008,0.067052,0.070934
